In [3]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

# تحميل البيانات
df = pd.read_csv("../data/cleaned_for_generation.csv")
df = df[["input", "output"]].fillna("").astype(str)

# تقسيم البيانات إلى تدريب واختبار
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
qa_pairs_train = train_df.to_dict(orient="records")
qa_pairs_test = test_df.to_dict(orient="records")



In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
qa_pairs_train = train_df.to_dict(orient="records")
qa_pairs_test = test_df.to_dict(orient="records")

In [5]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
index = faiss.read_index("../embeddings/faiss_index.bin")

C:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
test_questions = [pair["input"] for pair in qa_pairs_test]
test_answers = [pair["output"] for pair in qa_pairs_test]

In [7]:
train_qa = np.load("../embeddings/medical_questions.npy", allow_pickle=True)

In [21]:
import time

In [26]:
!pip install Rouge


In [30]:
from rouge import Rouge

In [49]:
def evaluate_retrieval(test_questions, test_answers, top_k=5):
    all_f1 = []
    all_precision = []
    all_recall = []
    all_rouge = []
    retrieved_answers = []
    
    # تشفير جميع أسئلة الاختبار مرة واحدة
    start_time = time.time()
    test_embeddings = model.encode(test_questions, batch_size=64)
    encode_time = time.time() - start_time
    
    rouge = Rouge()
    total_search_time = 0
    
    for i, (query_emb, true_answer) in enumerate(zip(test_embeddings, test_answers)):
        # البحث في الفهرس
        start_search = time.time()
        query_emb = np.array([query_emb]).astype("float32")
        distances, indices = index.search(query_emb, top_k)
        search_time = time.time() - start_search
        total_search_time += search_time
        
        # استرجاع الإجابات المرتبطة
        retrieved = [train_qa[idx]["output"] for idx in indices[0]]
        retrieved_answers.append(retrieved)
        
        # حساب F1-Score للإجابة الأولى
        true_tokens = set(true_answer.split())
        ret_tokens = set(retrieved[0].split())
        
        common_tokens = true_tokens & ret_tokens
        precision = len(common_tokens) / len(ret_tokens) if len(ret_tokens) > 0 else 0
        recall = len(common_tokens) / len(true_tokens) if len(true_tokens) > 0 else 0
        
        if (precision + recall) > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
        else:
            f1 = 0
            
        all_f1.append(f1)
        all_precision.append(precision)
        all_recall.append(recall)
        
        # حساب ROUGE score كل 100 عينة لتسريع العملية
        if i % 100 == 0:
            try:
                scores = rouge.get_scores(" ".join(retrieved[0].split()[:100]), 
                                        " ".join(true_answer.split()[:100]))
                all_rouge.append(scores[0]['rouge-l']['f'])
            except:
                all_rouge.append(0)
    
    # طباعة النتائج
    print("\n" + "="*60)
    print(f"النتائج على {len(test_questions)} عينة اختبار:")
    print("="*60)
    print(f"متوسط F1-Score: {np.mean(all_f1):.4f}")
    print(f"متوسط الدقة (Precision): {np.mean(all_precision):.4f}")
    print(f"متوسط الاستدعاء (Recall): {np.mean(all_recall):.4f}")
    print(f"متوسط ROUGE-L F1: {np.mean(all_rouge):.4f}")
    print("\nأوقات التنفيذ:")
    print(f"- وقت التشفير: {encode_time:.2f} ثانية")
    print(f"- متوسط وقت البحث لكل سؤال: {total_search_time/len(test_questions)*1000:.2f} مللي ثانية")
    print("="*60)
    
    # تقرير مفصل للعينة الأولى
    print("\nعينة اختبار توضيحية:")
    print(f"السؤال: {test_questions[0]}")
    print(f"الإجابة الصحيحة: {test_answers[0]}")
    print(f"أفضل إجابة مسترجعة: {retrieved_answers[0][0]}")
    print(f"F1-Score للعينة: {all_f1[0]:.4f}")
    
    return all_f1, all_precision, all_recall

In [51]:
f1_scores, precisions, recalls = evaluate_retrieval(test_questions, test_answers)


النتائج على 161695 عينة اختبار:
متوسط F1-Score: 0.9981
متوسط الدقة (Precision): 0.9982
متوسط الاستدعاء (Recall): 0.9983
متوسط ROUGE-L F1: 0.9987

أوقات التنفيذ:
- وقت التشفير: 216.11 ثانية
- متوسط وقت البحث لكل سؤال: 72.99 مللي ثانية

عينة اختبار توضيحية:
السؤال: سؤال: ما الفرق بين أدوية الربو و أدوية ؟ و ما الفرق بين الفينتولين و السيرتايد ؟ هل مادة السالبوتامول هيي الأساسية بغض النظر عن أسماء الشركات ؟ شكرا لكم
الإجابة الصحيحة: إجابة: هناك فرق نعم وقد يكون في ادوية قاسم مشترك بين الحالتين ولنستفيض بالشرح يجب ان يكون لديك خلفية صحية لتتمكن من المعرفة الاكاديمية للتفريق بين الحالتين وادويتهما
أفضل إجابة مسترجعة: إجابة: هناك فرق نعم وقد يكون في ادوية قاسم مشترك بين الحالتين ولنستفيض بالشرح يجب ان يكون لديك خلفية صحية لتتمكن من المعرفة الاكاديمية للتفريق بين الحالتين وادويتهما
F1-Score للعينة: 1.0000
